In [1]:
import sys; sys.path.insert(0,'..')

import os
import torch
import re
from fastai.vision.all import *
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchaudio import transforms
from data import NoisySpeechDataset
import time
from norse.models.dynamic_model import Autoencoder
import pdb
import matplotlib.pyplot as plt
from pypesq import pesq
import torch.nn.functional as F
import torchaudio
from tqdm.notebook import trange, tqdm
from IPython.display import Audio
from sampler import SpeechSampler

torchaudio.set_audio_backend('sox_io')

/home/dan/anaconda3/envs/norse/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
batch_size = 256

In [3]:
dataset = NoisySpeechDataset(base_dir='../data', window_size=2)
train_dl, val_dl = SpeechSampler(dataset, .2, batch_size)()
data = DataLoaders(train_dl, val_dl).cuda()

In [5]:
dataset[0]

[tensor([[-0.0066,  0.0004, -0.0028,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([[-0.0105, -0.0111, -0.0105,  ...,  0.0000,  0.0000,  0.0000]])]

In [19]:
Audio(dataset[0][0], rate = 16000)

In [20]:
Audio(dataset[0][1], rate = 16000)

In [21]:
Audio(net(dataset[0][0].unsqueeze(0).cuda()).cpu().detach().numpy()[0], rate = 16000)

In [4]:
net = Autoencoder(depth=7, kernel_size=32, upsample_type='transpose')

In [5]:
def SDRLoss(output, target):
    #import pdb;pdb.set_trace()
    output = output#.view(-1, 16384)
    target = target#.view(-1, 16384)
    # Check if this is ok
    dot_product = torch.sum(output * target)
    loss = (-1 * dot_product) / (torch.norm(target) * torch.norm(output))
    return loss

In [6]:
def weightedSDR(output, target):
    x = learn.xb[0]
    if x.shape != output.shape:
        output = F.pad(output, (0, 0, x.shape[-1] - output.shape[-1]))
    noise = x - target
    expected_noise = x - output 
    target_norm = torch.norm(target)
    target_norm_sq = target_norm * target_norm
    input_target_norm = torch.norm(x - target)
    input_target_norm_sq = input_target_norm * input_target_norm
    alpha = target_norm_sq / (target_norm_sq + input_target_norm_sq) 
    
    loss = alpha * SDRLoss(output, target) + (1 - alpha) * SDRLoss(noise, expected_noise)
    
    return loss

In [7]:
learn = Learner(data, net, loss_func=weightedSDR, cbs=[CudaCallback])

In [8]:
learn.lr_find()

torch.Size([256, 112, 234])
torch.Size([256, 112, 234])
torch.Size([256, 96, 484])
torch.Size([256, 96, 484])
torch.Size([256, 80, 984])
torch.Size([256, 80, 984])
torch.Size([256, 64, 1985])
torch.Size([256, 64, 1984])


RuntimeError: The size of tensor a (1985) must match the size of tensor b (1984) at non-singleton dimension 2

In [ ]:
lr = 1.4e-5
lrs = slice(lr/40,lr/4)
learn.fit_one_cycle(200, lrs)#, wd=1e-2, pct_start=0.2)

In [ ]:
learn.save('base_transp_7lay_1s')